In [1]:
import os
import pandas as pd

In [2]:
folder_path = 'Forecasts'  # Replace with your folder path
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


In [3]:
districts = ['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN']

In [4]:
dfs = []  # list to collect all df_req

for f in csv_files:
    df = pd.read_csv(f"{folder_path}/{f}")
    model = f.split('_')[0]
    
    try:
        df_req = df[['district', 'date', 'actual', 'forecast']]
    except KeyError:
        if model == 'bayesian':
            df_req = df[['district', 'date', 'actual', 'forecast_mean']]
            df_req = df_req.rename(columns={'forecast_mean': 'forecast'})  # optional: standardize column name
    df_req['model'] = model 
    dfs.append(df_req)  # collect the processed df_req

# After the loop, concatenate all collected dfs
final_df = pd.concat(dfs, ignore_index=True)

            
        
    # print(df_req.shape)

C:\Users\nauti\AppData\Local\Temp\ipykernel_18788\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model
C:\Users\nauti\AppData\Local\Temp\ipykernel_18788\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model


In [5]:
final_df.columns

Index(['district', 'date', 'actual', 'forecast', 'model'], dtype='object')

In [6]:
final_df = final_df[['model','district', 'date', 'actual', 'forecast']]

In [7]:
final_df['model'] = final_df['model'].str.upper()  # Convert district names to uppercase

In [8]:
final_df.columns

Index(['model', 'district', 'date', 'actual', 'forecast'], dtype='object')

In [9]:
import numpy as np
import pandas as pd

# Assuming 'final_df' already loaded, with columns: ['district', 'date', 'actual', 'forecast', 'model']

# Empty list to collect results
results = []

districts = final_df['district'].unique()
models = final_df['model'].unique()

for district in districts:
    for model in models:
        temp_df = final_df[(final_df['district'] == district) & (final_df['model'] == model)]
        if not temp_df.empty:
            rmse = np.sqrt(((temp_df['forecast'] - temp_df['actual'])**2).mean())
            results.append({
                'district': district,
                'model': model,
                'RMSE': rmse
            })

# Create DataFrame
rmse_df = pd.DataFrame(results)

# Sort if you want
rmse_df = rmse_df.sort_values(by=['district', 'RMSE'])

# See top results
print(rmse_df.head())


      district    model        RMSE
0   AHMEDNAGAR      ANN  669.757495
19  AHMEDNAGAR  XGBOOST  675.055056
5   AHMEDNAGAR       ET  715.481323
3   AHMEDNAGAR       DT  720.184856
15  AHMEDNAGAR       RF  809.974570


In [10]:
rmse_df[rmse_df.district == "THANE"].sort_values(by='RMSE', ascending=True)

,district,model,RMSE
633,THANE,LINEAR,2419.483607
625,THANE,ELASTICNET,2419.493796
629,THANE,HUBER,2516.256146
631,THANE,LASSO,2560.953500
636,THANE,RIDGE,2560.972327
638,THANE,TBATS,2726.684249
627,THANE,EXP,2727.980156
637,THANE,SARIMA,2740.067308
622,THANE,ARIMA,2740.067308
635,THANE,RF,2870.719049


In [11]:
final_df['model'].unique()

array(['ANN', 'ARIMA', 'BAYESIAN', 'DT', 'ELASTICNET', 'ET', 'EXP', 'GB',
       'HUBER', 'KNN', 'LASSO', 'LGBM', 'LINEAR', 'LSTM', 'PROPHET', 'RF',
       'RIDGE', 'SARIMA', 'TBATS', 'XGBOOST'], dtype=object)

In [12]:
pivot_df = rmse_df.pivot(index='district', columns='model', values='RMSE')
print(pivot_df)

model                    ANN        ARIMA     BAYESIAN            DT  \
district                                                               
AHMEDNAGAR        669.757495  1085.764859  1147.891774    720.184856   
AKOLA                    NaN   352.227870   640.715349    802.948150   
AMRAVATI                 NaN   909.010384   527.651504    785.179135   
AURANGABAD               NaN   323.389260   780.306644   1644.231845   
BEED              397.369238   624.397807   442.698264    643.315675   
BHANDARA                 NaN   306.393864   202.391842    346.467154   
BULDHANA                 NaN   491.118764   335.927304    357.587825   
CHANDRAPUR        325.574320   579.511060   363.024110    463.558040   
DHULE             389.953024   366.238032   337.579786    298.564373   
GADCHIROLI        188.185448   279.218066   174.882582    192.259094   
GONDIA            223.214068   222.071523   202.753151    192.839888   
HINGOLI           124.834672   174.280234   130.133429    137.00

In [13]:
pivot_df.to_csv('I55_rmse_pivot.csv', index=True)

In [14]:
# Get the best model for each district (lowest RMSE)
best_models = rmse_df.loc[rmse_df.groupby('district')['RMSE'].idxmin()]

# Display the result
best_models[['district', 'model', 'RMSE']].to_csv('I55_best_models.csv', index=False)